# **MolSSI API Demo - Carboxylic Acid Library**

The full API documentation for the carboxylic acid library can be found [here](https://descriptor-libraries.molssi.org/api/acids/docs#/current/).

In [ ]:
# First, lets import the packages we will need.

import pandas as pd
import requests    # this is not part of the standard library, so you may need to install it first (e.g, pip install requests)

## Downloading data for a single molecule

For this you need to know the id of the molecule on the MolSSI database. For example, lets search for 4-fluorobenzoic acid which has the molecule id `Ac-ADX-47273`.

<img src="./img/acid.png" alt="acid" width="400"/>

In [ ]:
# To make things as re-usable as possible, lets define the molecule id as a variable. Note this has to be a string!
molecule_id = "Ac-ADX-47273"

In [ ]:
# We can use a standard URL string to access the API and retrieve the DFT features that we want for this molecule.
url = f"https://descriptor-libraries.molssi.org/api/acids/molecules/data/export/{molecule_id}?data_type=dft"

In [ ]:
# Load the data into a pandas dataframe
df = pd.read_csv(url)

In [ ]:
display(df)

## Downloading structures for a single molecule

Again, we need to know the id for our molecule of interest (below).

<img src="./img/acid2.png" alt="acid2" width="400"/>

In [ ]:
molecule_id = "Ac1558"

This molecule has a total of **8 conformers**, all which have a unique conformer id in the database. We can retrieve this information as follows:

In [ ]:
# Again, we need to set up a API URL to access the data we want.
url = f"https://descriptor-libraries.molssi.org/api/acids/molecules/{molecule_id}"

In [ ]:
# This time the data is returned as a json object, so we can use the requests package to access it.
response = requests.get(url)
data = response.json()

In [ ]:
data

Lets download all of the conformers as SDFs to a folder called "molecules". Make sure that you have made this folder before you run the next cell!

In [ ]:
# Set up a loop to download each conformer
for conformer in data['conformers_id']:
    url = f"https://descriptor-libraries.molssi.org/api/acids/conformers/export/sdf/{conformer}"
    response = requests.get(url)
    with open(f"molecules/{conformer}.sdf", "wb") as file:
        file.write(response.content)

## Download a batch of DFT properties from a reaction dataset I have collected

We will be using an amide coupling rate dataset collected by the Sigman lab and co-workers in 2022 (`haas2022.csv` in the datasets directory). This is modeled using DFT features from both carboxylic acids and primary amines.

Citation: [https://doi.org/10.1073/pnas.2118451119](https://doi.org/10.1073/pnas.2118451119)

<img src="./img/haas2022.png" alt="amide-coupling" width="400"/>

In [ ]:
# First we need to load in the data
amide_data = pd.read_csv("./datasets/haas2022.csv", index_col=0)
display(amide_data.head())

In [ ]:
# Extract the unique acid and amine IDs
acid_ids = amide_data['Acid'].unique()
amine_ids = amide_data['Amine'].unique()

print(acid_ids)
print(amine_ids)

In [ ]:
# Batch download the acid DFT features using the MolSSI API
batch_url = f"https://descriptor-libraries.molssi.org/api/acids/molecules/data/export/batch?molecule_ids={','.join(acid_ids)}&data_type=dft"

In [ ]:
acids_descriptors = pd.read_csv(batch_url, index_col=0)
# add "acid" prefix to column names
acids_descriptors.columns = [f"acid_{col}" for col in acids_descriptors.columns]

display(acids_descriptors.head())

In [ ]:
# And the same thing for the amines - NOTE the changes to the URL to access the amines API
batch_url = f"https://descriptor-libraries.molssi.org/api/primary-amines/molecules/data/export/batch?molecule_ids={','.join(amine_ids)}&data_type=dft"

In [ ]:
amine_descriptors = pd.read_csv(batch_url, index_col=0)
# add "amine" prefix to column names
amine_descriptors.columns = [f"amine_{col}" for col in amine_descriptors.columns]

display(amine_descriptors.head())

In [ ]:
# We are just interested in the Boltzmann-weighted features, so we can filter the dataframes to only include "boltz"
acids_descriptors = acids_descriptors.filter(like='boltz')
amine_descriptors = amine_descriptors.filter(like='boltz')

# Remove any boltz_std columns
acids_descriptors = acids_descriptors[[col for col in acids_descriptors.columns if 'std' not in col]]
amine_descriptors = amine_descriptors[[col for col in amine_descriptors.columns if 'std' not in col]]

In [ ]:
acids_descriptors.head()

In [ ]:
amine_descriptors.head()

In [ ]:
# Combine the acid and amine descriptors with the original amide_data dataframe
amide_data = amide_data.join(acids_descriptors, on='Acid')
amide_data = amide_data.join(amine_descriptors, on='Amine')

In [ ]:
amide_data.head()

In [ ]:
# Finally, we can save the combined dataframe so we can use it with ROBERT.
amide_data.to_csv('amide_data_robert.csv')

## Use ROBERT to make an amide couple rate data multivariate linear regression (MLR) model.

In [ ]:
! python -m robert --csv_name "amide_data_robert.csv" --names "Coupling" --y "ln rate" --discard "["Acid","Amine","rate"]" --model "MVL" --split "KN"